# CNN image prediction
### Forward propagation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms

torch.set_printoptions(linewidth=120)

In [2]:
train_set = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=10)

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)     #outchanels = number of filters/ arbitrarily chosen
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)          # the 12 came from out_channels * 4(height) * 4(width) ***/image.flatten(dim=1).shape toget 120**
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):

        t.size(0)
        t = t
 
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)  #fc is fully connected layers or linear layers or dense layer/densenet

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))       # 4 * 4 height and width of the output channels
        t = F.relu(self.fc2(t))

        t = self.out(t)
        

        return t


- 4*4 means output of the last CNN is 4x4 image/filter:
- output_Size_Of_Cov = [(inputSize + 2*padding - filterSize)/stride] + 1
- inputSize=12(based on the last out_channels?), pad = 0, filterSize=5 and stride=1, so (12+2*0-5)/1+1=8. 
-
- tensor=(1,28,28), kernel size (5,5) 
- formula--> 
- 1.for convolution part-->{tensor_size - kernel_size+1}.
- 2.for pooling part--> tensor_size/pooling_size i.e (24,24)-->pooled with 2 size--> (12,12)
- Notations for kernel=[ ], for image=( ) 
- Now lets start
- (1,28,28)-->[5,5] with 6 filters-->(6,24,24)-->pooling_size=2--->(6,12,12)--->[5,5] with 12 filter--->(12,8,8)--->pooling_size=2--->(12,4,4)

- And these final is flattened to 12\*4\*4




In [4]:
torch.set_grad_enabled(False)             # disable gradient tracking

In [5]:
sample = next(iter(train_set))

In [6]:
image, label = sample
image.shape

torch.Size([1, 28, 28])

In [7]:
image.unsqueeze(0).shape         # gives a batch with size of 1

torch.Size([1, 1, 28, 28])

In [24]:
image.flatten(start_dim=1).shape


torch.Size([1, 784])

In [8]:
network = Network()

In [9]:
network

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [10]:
pred = network(image.unsqueeze(0))       # gets the image prediction


In [11]:
pred.shape

torch.Size([1, 10])

In [12]:
pred

tensor([[-0.0187, -0.0438, -0.0600, -0.1338, -0.0996, -0.0013, -0.0730, -0.1243,  0.0407, -0.0778]])

In [13]:
label

9

In [14]:
pred.argmax(dim=1)

tensor([8])

In [15]:
F.softmax(pred, dim=1)

tensor([[0.1040, 0.1014, 0.0998, 0.0927, 0.0959, 0.1058, 0.0985, 0.0936, 0.1104, 0.0980]])

In [16]:
F.softmax(pred, dim=1).sum()

tensor(1.)

In [17]:
net1 = Network()

In [18]:
net1(image.unsqueeze(0))

tensor([[ 0.0282, -0.0957, -0.0677,  0.0921,  0.0495,  0.0043,  0.1716,  0.1346, -0.0628,  0.0588]])

In [19]:
net2 = Network()

In [20]:
net2(image.unsqueeze(0))

tensor([[-0.0032, -0.1487,  0.1551,  0.0248, -0.0172, -0.1361, -0.1260,  0.1023,  0.0901, -0.0191]])